**MILESTONE-1**

In [ ]:
import easyocr
import cv2
import pandas as pd
from IPython.display import display, clear_output

In [ ]:
csv_path = "diabetes.csv" 
df = pd.read_csv(csv_path)  

In [ ]:
print("Diabetes Dataset (First 5 Rows):")
display(df.head())

In [ ]:
# Load image
image_path = "medical_report.png"
image = cv2.imread(image_path)

In [ ]:
reader = easyocr.Reader(['en'])
results = reader.readtext(image)

In [ ]:
print("\nExtracted Text from Medical Report:\n")
for bbox, text, confidence in results:
    print(f"{text} (Confidence: {confidence:.2f})")

**MILESTONE-2**

**XG BOOST**

In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
df = pd.read_csv('diabetes.csv')
print("Diabetes Dataset (First 5 Rows):")
display(df.head())

cols_to_fix = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in cols_to_fix:
    median_val = df[df[col] != 0][col].median()
    df[col] = df[col].replace(0, median_val)

In [ ]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
model = XGBClassifier(
    n_estimators=100,
    max_depth=3,           
    learning_rate=0.05,    
    reg_alpha=1,           
    reg_lambda=1.5,       
    subsample=0.7,         
    colsample_bytree=0.7,  
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

model.fit(X_train, y_train)

In [ ]:
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

print(f"Training Accuracy: {accuracy_score(y_train, train_preds) * 100:.2f}%")
print(f"Testing Accuracy:  {accuracy_score(y_test, test_preds) * 100:.2f}%")
print("\n--- Classification Report ---")
print(classification_report(y_test, test_preds))

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

style = {'description_width': 'initial'}

# Inputs for all 8 features
inputs = {
    'Pregnancies': widgets.IntText(value=0, description="Pregnancies:", style=style),
    'Glucose': widgets.IntText(value=100, description="Glucose (mg/dL):", style=style),
    'BloodPressure': widgets.IntText(value=70, description="Blood Pressure:", style=style),
    'SkinThickness': widgets.IntText(value=20, description="Skin Thickness:", style=style),
    'Insulin': widgets.IntText(value=80, description="Insulin:", style=style),
    'BMI': widgets.FloatText(value=25.0, description="BMI:", style=style),
    'DPF': widgets.FloatText(value=0.47, description="Pedigree Function:", style=style),
    'Age': widgets.IntText(value=30, description="Age:", style=style)
}

btn = widgets.Button(description="Check Alerts", button_style='warning')
output = widgets.Output()

#THRESHOLD ALERT FUNCTION
def check_thresholds(b):
    with output:
        clear_output()
        print("="*35)
        print("       MEDICAL RISK ALERTS      ")
        print("="*35)
        
        any_alert = False

        # Glucose Logic
        if inputs['Glucose'].value >= 126:
            print("❌ ALERT: High Glucose (Diabetic)")
            any_alert = True
        elif inputs['Glucose'].value >= 100:
            print("⚠️ WARNING: Elevated Glucose (Pre-diabetic)")
            any_alert = True

        # BMI Logic
        if inputs['BMI'].value >= 30:
            print("❌ ALERT: Obesity Detected")
            any_alert = True
        elif inputs['BMI'].value >= 25:
            print("⚠️ WARNING: Overweight")
            any_alert = True

        # Blood Pressure Logic
        if inputs['BloodPressure'].value >= 90:
            print("❌ ALERT: High Blood Pressure")
            any_alert = True

        if not any_alert:
            print("✅ Clinical values are within normal limits.")
        
        print("="*35)
        print("Note: Click again after updating values.")


btn.on_click(check_thresholds)
ui_layout = widgets.VBox(list(inputs.values()) + [btn])
display(ui_layout, output)

**MILESTONE-3**

**RULE-BASED NLP**

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import re


In [ ]:
# Diet rules
RULES = {
    "avoid sugar": {
        "tag": "Avoid Sugar",
        "Breakfast": ["Vegetable omelette", "Whole-grain toast", "Unsweetened tea"],
        "Lunch": ["Brown rice", "Dal", "Fresh vegetable salad (no sweets)"],
        "Dinner": ["Grilled paneer/chicken", "Steamed vegetables"],
        "Snacks": ["Roasted chana", "Nuts"]
    },
    "reduce salt": {
        "tag": "Reduce Salt",
        "Lunch": ["Low-sodium dal", "Steamed vegetables"],
        "Dinner": ["Grilled protein", "Clear vegetable soup (no added salt)"]
    },
    "eat leafy": {
        "tag": "Increase Leafy Vegetables",
        "Lunch": ["Palak dal", "Brown rice", "Cucumber salad"],
        "Dinner": ["Spinach sauté", "Paneer/chicken"]
    },
    "high fiber": {
        "tag": "High Fiber Diet",
        "Breakfast": ["Oats", "Apple", "Flaxseed"],
        "Snacks": ["Fruit bowl", "Soaked almonds"]
    },
    "eat carbohydrates": {
        "tag": "Include Carbohydrates",
        "Breakfast": ["Oats", "Whole grain toast"],
        "Lunch": ["Brown rice", "Roti", "Whole grain pasta"],
        "Dinner": ["Quinoa", "Brown rice", "Vegetables"],
        "Snacks": ["Whole-grain crackers", "Fruits"]
    },
    "increase protein": {
        "tag": "Increase Protein Intake",
        "Breakfast": ["Eggs", "Paneer"],
        "Lunch": ["Grilled chicken", "Dal", "Lentils"],
        "Dinner": ["Paneer", "Tofu", "Fish"],
        "Snacks": ["Nuts", "Greek yogurt"]
    }
}


In [ ]:
# Keywords mapping for detection
KEYWORDS = {
    "avoid sugar": [
        "avoid sugar", "no sugar", "skip sweets", "avoid drinks",
        "refrain from sugar", "refrain from sweets", "do not eat sugar",
        "avoid sweet foods", "no sweets", "cut down sugar","processed sugars"
    ],
    "reduce salt": [
        "reduce salt", "low salt", "avoid salt", "limit salt",
        "cut down salt", "less sodium", "reduce sodium"
    ],
    "eat leafy": [
        "eat leafy", "leafy vegetables", "more greens", "spinach",
        "palak", "eat vegetables", "include vegetables", "take greens",
        "green vegetables", "consume vegetables"
    ],
    "high fiber": [
        "high fiber", "fiber rich", "oats", "bran", "take fiber",
        "eat fiber", "increase fiber", "increase fiber intake", "fiber foods"
    ],
    "eat carbohydrates": [
        "eat carbohydrates", "carbs", "eat carbs", "rice", "roti",
        "pasta", "whole grains", "take carbs", "include carbs"
    ],
    "increase protein": [
        "increase protein", "eat protein", "protein", "chicken",
        "paneer", "eggs", "dal", "tofu", "fish", "legumes",
        "take protein", "prioritize protein", "protein foods"
    ]
}


In [ ]:
# Extract rules robustly using regex word boundaries
def extract_rules(text):
    text_lower = text.lower()
    found_rules = []

    for rule, keywords in KEYWORDS.items():
        for kw in keywords:
            if re.search(r'\b' + re.escape(kw) + r'\b', text_lower):
                found_rules.append(rule)
                break  
    return list(dict.fromkeys(found_rules))

In [ ]:
# Generate diet plan

def generate_diet_plan(found_rules):
    plan = {
        "Morning": [],
        "Lunch": [],
        "Evening": [],
        "Night": []
    }

    for rule in found_rules:
        rule_plan = RULES[rule]
        for meal, items in rule_plan.items():
            if meal == "Breakfast":
                plan["Morning"].extend(items)
            elif meal == "Lunch":
                plan["Lunch"].extend(items)
            elif meal == "Snacks":
                plan["Evening"].extend(items)
            elif meal == "Dinner":
                plan["Night"].extend(items)
    for meal in plan:
        plan[meal] = list(dict.fromkeys(plan[meal]))

    return plan


In [ ]:
# Widgets
text_input = widgets.Textarea(
    placeholder="Paste doctor's prescription text here...",
    layout=widgets.Layout(width='100%', height='140px')
)

button = widgets.Button(description="Generate Diet Plan", button_style='success')
output = widgets.Output()

def on_click(b):
    with output:
        clear_output()
        text = text_input.value.strip()

        if not text:
            print("Please enter prescription text.")
            return

        found_rules = extract_rules(text)

        print("🧾 Doctor's Instructions Found:\n")
        if found_rules:
            for i, r in enumerate(found_rules, 1):
                print(f"{i}. {RULES[r]['tag']}")
        else:
            print("⚠️ No diet instructions detected.")

        print("\n" + "="*55)
        print("🩺 PROFESSIONAL DIET PLAN \n")

        plan = generate_diet_plan(found_rules)

        print("🍽 Morning (8:00 AM)")
        for item in plan["Morning"]:
            print(f"- {item}")

        print("\n🍛 Lunch (1:00 PM)")
        for item in plan["Lunch"]:
            print(f"- {item}")

        print("\n☕ Evening (4:30 PM)")
        for item in plan["Evening"]:
            print(f"- {item}")

        print("\n🌙 Night (7:30 PM)")
        for item in plan["Night"]:
            print(f"- {item}")

        print("\n💧 Hydration: 2.5-3 liters water daily.")

button.on_click(on_click)
display(text_input, button, output)